#  HKU CIVL7024  Assignment 2

## Name:YAN,Yiting Uni. No.:3036494960

## Part I Image Processing
The goal of this part of the assignment is to stitch the two images together.(This notebook (Part I) requires the `tung_ping_chau_view1.jpeg` and `tung_ping_chau_view2.jpeg` files to be placed in the same directory level.)

1.Load the images and convert to grayscale. Explain, in the notebook, why do we want to do so.

In [ ]:
!pip install opencv-python opencv-contrib-python matplotlib numpy


In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import warnings
warnings.filterwarnings('ignore')

# Load the images
img1 = cv2.imread('tung_ping_chau_view1.jpeg')
img2 = cv2.imread('tung_ping_chau_view2.jpeg')

# Check if images are loaded correctly
if img1 is None:
    print("Error: Could not load 'tung_ping_chau_view1.jpeg'")
    # Try alternative filename
    img1 = cv2.imread('tung_ping_Chau_view1.jpg')

if img2 is None:
    print("Error: Could not load 'tung_ping_chau_view2.jpeg'")
    # Try alternative filename
    img2 = cv2.imread('tung_ping_Chau_view2.jpg')

# Convert to grayscale
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Display both original and grayscale images
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Display original images
axes[0, 0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Image 1 (Original)')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Image 2 (Original)')
axes[0, 1].axis('off')

# Display grayscale images
axes[1, 0].imshow(gray1, cmap='gray')
axes[1, 0].set_title('Image 1 (Grayscale)')
axes[1, 0].axis('off')

axes[1, 1].imshow(gray2, cmap='gray')
axes[1, 1].set_title('Image 2 (Grayscale)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Print confirmation
print(f"Original Image 1 shape: {img1.shape}")
print(f"Grayscale Image 1 shape: {gray1.shape}")
print(f"Original Image 2 shape: {img2.shape}")
print(f"Grayscale Image 2 shape: {gray2.shape}")


## Why Convert to Grayscale?

1. **Computational Efficiency**：Grayscale images have only one channel, while color images have three channels (RGB). Processing grayscale images requires only one-third of the computational load compared to color images.

2. **Feature Detection Effectiveness**：Most feature detection algorithms (such as Harris, SIFT, and SURF) are designed for single-channel images. Grayscale images contain brightness information, which is crucial for edge and corner detection.

3. **Reduced Noise Impact**：Color information may introduce additional noise. Converting to grayscale simplifies the image, making features more stable.

4. **Algorithm Compatibility**：Many feature detection functions in OpenCV require input in grayscale format.

2.Use the Harris detector to detect the corner points. Writing your own detector is recommended as good practice, but using the OpenCV library is also fine.

In [ ]:
# Harris Corner Detection (Custom Implementation)
def harris_corner_detector(image, k=0.04, threshold_ratio=0.01):
    """
    Custom Harris Corner Detector

    Parameters:
    image: Input image
    k: Harris detector free parameter (typically 0.04-0.06)
    threshold_ratio: Corner response threshold ratio (0.01 = 1% of max)

    Returns:
    corners: Binary image with corners marked
    corner_coords: Coordinates of detected corners (limited to 500)
    """
    # 1. Calculate gradients in x and y directions
    Ix = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # 2. Calculate gradient products
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # 3. Apply Gaussian filtering to weight gradient products
    kernel_size = 3
    sigma = 1.5
    Ixx = cv2.GaussianBlur(Ixx, (kernel_size, kernel_size), sigma)
    Iyy = cv2.GaussianBlur(Iyy, (kernel_size, kernel_size), sigma)
    Ixy = cv2.GaussianBlur(Ixy, (kernel_size, kernel_size), sigma)

    # 4. Calculate Harris response function
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace ** 2)

    # 5. Normalize response to 0-255
    R_norm = cv2.normalize(R, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # 6. Thresholding
    threshold_value = threshold_ratio * 255
    corners_binary = np.zeros_like(image, dtype=np.uint8)
    corners_binary[R_norm > threshold_value] = 255

    # 7. Non-maximum suppression (using dilation to find local maxima)
    # Create a kernel for dilation
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(R_norm, kernel)

    # Local maxima are points where the original value equals the dilated value
    local_maxima = (R_norm == dilated)

    # Combine threshold and local maxima
    corners_final = np.zeros_like(image, dtype=np.uint8)
    corners_final[(R_norm > threshold_value) & local_maxima] = 255

    # 8. Extract corner coordinates
    corner_coords = np.column_stack(np.where(corners_final > 0))

    # 9. Sort corners by response strength (descending)
    if len(corner_coords) > 0:
        # Get response values for each corner
        responses = R_norm[corner_coords[:, 0], corner_coords[:, 1]]
        # Sort by response (descending)
        sorted_indices = np.argsort(responses)[::-1]
        corner_coords = corner_coords[sorted_indices]

    # Return only first 500 corners (or all if less than 500)
    max_corners = min(500, len(corner_coords))

    return corners_final, corner_coords[:max_corners]

# Apply Harris detector with adjusted parameters
harris1, corners1 = harris_corner_detector(gray1, k=0.05, threshold_ratio=0.02)
harris2, corners2 = harris_corner_detector(gray2, k=0.05, threshold_ratio=0.02)

print(f"Number of Harris corners detected in image 1: {len(corners1)}")
print(f"Number of Harris corners detected in image 2: {len(corners2)}")

# Display Harris detection results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Image 1 with corners
axes[0, 0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
if len(corners1) > 0:
    axes[0, 0].scatter(corners1[:, 1], corners1[:, 0], s=10, c='r', marker='o', alpha=0.6)
axes[0, 0].set_title(f'Image 1 with Harris Corners ({len(corners1)} corners)')
axes[0, 0].axis('off')

# Harris response 1
axes[0, 1].imshow(harris1, cmap='hot')
axes[0, 1].set_title('Harris Response (Image 1)')
axes[0, 1].axis('off')

# Image 2 with corners
axes[1, 0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
if len(corners2) > 0:
    axes[1, 0].scatter(corners2[:, 1], corners2[:, 0], s=10, c='r', marker='o', alpha=0.6)
axes[1, 0].set_title(f'Image 2 with Harris Corners ({len(corners2)} corners)')
axes[1, 0].axis('off')

# Harris response 2
axes[1, 1].imshow(harris2, cmap='hot')
axes[1, 1].set_title('Harris Response (Image 2)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Optional: Show corner statistics
if len(corners1) > 0:
    print(f"\nImage 1 corner statistics:")
    print(f"  - Min corner coordinates: ({corners1[:, 0].min()}, {corners1[:, 1].min()})")
    print(f"  - Max corner coordinates: ({corners1[:, 0].max()}, {corners1[:, 1].max()})")

if len(corners2) > 0:
    print(f"\nImage 2 corner statistics:")
    print(f"  - Min corner coordinates: ({corners2[:, 0].min()}, {corners2[:, 1].min()})")
    print(f"  - Max corner coordinates: ({corners2[:, 0].max()}, {corners2[:, 1].max()})")

3.Use the OpenCV library to extract keypoints and compute descriptors through the function SIFT_create.detectAndCompute(). Output the number of keypoints in each image.
You can find more details in the OpenCV documents. Note: To be efficient, we use no more than 5,000 keypoints in this case.

In [ ]:
# Use SIFT to extract keypoints and descriptors
print("="*60)
print("STEP 3: SIFT Feature Extraction")
print("="*60)

# Initialize the SIFT detector, limiting the number of keypoints for improved efficiency
sift = cv2.SIFT_create(nfeatures=5000,
                       nOctaveLayers=3,
                       contrastThreshold=0.04,
                       edgeThreshold=10)

# Detect keypoints and compute descriptors
print("Extracting SIFT features from Image 1...")
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
print("Extracting SIFT features from Image 2...")
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

print(f"\nNumber of SIFT keypoints in image 1: {len(keypoints1)}")
print(f"Number of SIFT keypoints in image 2: {len(keypoints2)}")

# Display keypoint statistical information
if len(keypoints1) > 0:
    scales1 = [kp.size for kp in keypoints1]
    angles1 = [kp.angle for kp in keypoints1]
    print(f"\nImage 1 Keypoint Statistics:")
    print(f"  - Average scale: {np.mean(scales1):.2f}")
    print(f"  - Scale range: [{min(scales1):.2f}, {max(scales1):.2f}]")
    print(f"  - Average angle: {np.mean(angles1):.1f}°")

if len(keypoints2) > 0:
    scales2 = [kp.size for kp in keypoints2]
    angles2 = [kp.angle for kp in keypoints2]
    print(f"\nImage 2 Keypoint Statistics:")
    print(f"  - Average scale: {np.mean(scales2):.2f}")
    print(f"  - Scale range: [{min(scales2):.2f}, {max(scales2):.2f}]")
    print(f"  - Average angle: {np.mean(angles2):.1f}°")

# Draw keypoints (only display the first 100 to avoid overcrowding)
img1_keypoints = cv2.drawKeypoints(img1, keypoints1[:100], None,
                                   flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_keypoints = cv2.drawKeypoints(img2, keypoints2[:100], None,
                                   flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
axes[0].imshow(cv2.cvtColor(img1_keypoints, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'Image 1: {len(keypoints1)} SIFT Keypoints')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img2_keypoints, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Image 2: {len(keypoints2)} SIFT Keypoints')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Display keypoint orientation distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

if len(keypoints1) > 0:
    angles1 = [kp.angle for kp in keypoints1]
    axes[0].hist(angles1, bins=36, range=(0, 360), alpha=0.7, color='blue')
    axes[0].set_xlabel('Orientation (degrees)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Keypoint Orientations - Image 1')
    axes[0].grid(True, alpha=0.3)

if len(keypoints2) > 0:
    angles2 = [kp.angle for kp in keypoints2]
    axes[1].hist(angles2, bins=36, range=(0, 360), alpha=0.7, color='red')
    axes[1].set_xlabel('Orientation (degrees)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Keypoint Orientations - Image 2')
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

4.Match the keypoints using the BFMatch() function in OpenCV.

In [ ]:
# Match Keypoints
print("="*60)
print("STEP 4: Feature Matching using BFMatcher")
print("="*60)

# Create BFMatcher object (using L2 distance and cross-checking)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match descriptors
print("Matching features between images...")
matches = bf.match(descriptors1, descriptors2)

# Sort by distance (smaller distance indicates better match)
matches = sorted(matches, key=lambda x: x.distance)

print(f"\nTotal number of matches found: {len(matches)}")
print(f"Best match distance: {matches[0].distance:.2f}")
print(f"Worst match distance: {matches[-1].distance:.2f}")
print(f"Median match distance: {matches[len(matches)//2].distance:.2f}")
print(f"Average match distance: {np.mean([m.distance for m in matches]):.2f}")

# Analyze match distance distribution
match_distances = [m.distance for m in matches]
percentiles = [25, 50, 75, 90]
percentile_values = np.percentile(match_distances, percentiles)
print("\nMatch distance percentiles:")
for p, v in zip(percentiles, percentile_values):
    print(f"  {p}th percentile: {v:.2f}")

# Draw the top 50 best matches
print("\nDisplaying top 50 matches...")
matched_img = cv2.drawMatches(img1, keypoints1, img2, keypoints2,
                              matches[:50], None,
                              flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

fig, axes = plt.subplots(1, 1, figsize=(18, 9))
axes.imshow(cv2.cvtColor(matched_img, cv2.COLOR_BGR2RGB))
axes.set_title('Top 50 Feature Matches')
axes.axis('off')
plt.tight_layout()
plt.show()

# Display match distance distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(match_distances, bins=50, alpha=0.7, color='green')
ax1.set_xlabel('Match Distance (lower is better)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Match Distances')
ax1.grid(True, alpha=0.3)

# Cumulative Distribution Function
sorted_distances = np.sort(match_distances)
y_vals = np.arange(1, len(sorted_distances)+1) / len(sorted_distances)
ax2.plot(sorted_distances, y_vals, linewidth=2, color='blue')
ax2.set_xlabel('Match Distance')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Cumulative Distribution of Match Distances')
ax2.grid(True, alpha=0.3)

# Mark key percentile points
for p, v in zip(percentiles, percentile_values):
    ax2.axvline(x=v, color='red', linestyle='--', alpha=0.5)
    ax2.text(v, 0.95-p/100, f'{p}%', fontsize=9, color='red')

plt.tight_layout()
plt.show()

5.Compute the homography from the matched points using RANSAC. Print the Homography Matrix and explain in the notebook how RANSAC works here.

In [ ]:
# Compute Homography Matrix using RANSAC


print("="*60)
print("STEP 5: Compute Homography using RANSAC")
print("="*60)

# Prepare coordinates of matched points
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

print(f"Number of matched point pairs: {len(src_pts)}")

# Compute Homography Matrix using RANSAC
# H maps dst_pts to src_pts: src_pts = H * dst_pts
print("\nRunning RANSAC to estimate homography matrix...")
H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC,
                             ransacReprojThreshold=5.0,
                             maxIters=2000,
                             confidence=0.995)

print("\n" + "="*40)
print("Homography Matrix (H):")
print("="*40)
print(f"[[{H[0,0]:.6f}, {H[0,1]:.6f}, {H[0,2]:.6f}]")
print(f" [{H[1,0]:.6f}, {H[1,1]:.6f}, {H[1,2]:.6f}]")
print(f" [{H[2,0]:.6f}, {H[2,1]:.6f}, {H[2,2]:.6f}]]")

# Interpret the homography matrix
print("\n" + "="*40)
print("Homography Matrix Interpretation:")
print("="*40)
print("A homography matrix H (3x3) transforms points from image 2 to image 1:")
print("  [x1]   [h11 h12 h13] [x2]")
print("  [y1] = [h21 h22 h23] [y2]")
print("  [w ]   [h31 h32 h33] [1 ]")
print("\nWhere (x2, y2) is a point in image 2,")
print("      (x1/w, y1/w) is the corresponding point in image 1,")
print("      w = h31*x2 + h32*y2 + h33 (for normalization)")

# RANSAC statistics
inlier_count = np.sum(mask)
outlier_count = len(mask) - inlier_count
inlier_ratio = inlier_count / len(mask)

print("\n" + "="*40)
print("RANSAC Results:")
print("="*40)
print(f"Number of inliers (good matches): {inlier_count}")
print(f"Number of outliers (bad matches): {outlier_count}")
print(f"Inlier ratio: {inlier_ratio:.2%}")
print(f"RANSAC reprojection threshold: 5.0 pixels")

# Draw inlier and outlier matches
matches_mask = mask.ravel().tolist()

# Plot match graph (green for inliers, red for outliers)
draw_params = dict(matchColor=(0, 255, 0),  # 绿色表示inliers
                   singlePointColor=None,
                   matchesMask=matches_mask[:100],  # 只显示前100个匹配的状态
                   flags=2)

matched_img_with_inliers = cv2.drawMatches(img1, keypoints1, img2, keypoints2,
                                           matches[:100], None, **draw_params)

fig, axes = plt.subplots(1, 1, figsize=(18, 9))
axes.imshow(cv2.cvtColor(matched_img_with_inliers, cv2.COLOR_BGR2RGB))
axes.set_title(f'Matches: Green = Inliers ({inlier_count} total), Red = Outliers ({outlier_count} total)')
axes.axis('off')
plt.tight_layout()
plt.show()

# Visualize distribution of inliers
inlier_matches = [m for i, m in enumerate(matches) if matches_mask[i] == 1]
outlier_matches = [m for i, m in enumerate(matches) if matches_mask[i] == 0]

print(f"\nInlier match statistics:")
if inlier_matches:
    inlier_distances = [m.distance for m in inlier_matches]
    print(f"  - Average distance: {np.mean(inlier_distances):.2f}")
    print(f"  - Min distance: {min(inlier_distances):.2f}")
    print(f"  - Max distance: {max(inlier_distances):.2f}")

if outlier_matches:
    outlier_distances = [m.distance for m in outlier_matches]
    print(f"\nOutlier match statistics:")
    print(f"  - Average distance: {np.mean(outlier_distances):.2f}")
    print(f"  - Min distance: {min(outlier_distances):.2f}")
    print(f"  - Max distance: {max(outlier_distances):.2f}")

# Visualize distance distribution of inliers and outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if inlier_matches and outlier_matches:
    inlier_distances = [m.distance for m in inlier_matches]
    outlier_distances = [m.distance for m in outlier_matches]

    axes[0].boxplot([inlier_distances, outlier_distances], labels=['Inliers', 'Outliers'])
    axes[0].set_ylabel('Match Distance')
    axes[0].set_title('Distance Comparison: Inliers vs Outliers')
    axes[0].grid(True, alpha=0.3)

    axes[1].hist([inlier_distances, outlier_distances], bins=30,
                 label=['Inliers', 'Outliers'], alpha=0.7)
    axes[1].set_xlabel('Match Distance')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distance Distribution: Inliers vs Outliers')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## How RANSAC Works for Homography Estimation

**RANSAC (Random Sample Consensus)** is a robust algorithm for estimating model parameters from data containing outliers. Here's how it works for homography estimation:

### Step-by-Step Process:

1. **Random Sampling**:
   - Randomly select 4 point pairs from all matches (minimum needed to compute a homography)
   - These form a "hypothetical inlier set"

2. **Model Estimation**:
   - Compute a homography matrix H using these 4 point pairs
   - This is a "hypothesis model"

3. **Model Verification**:
   - Apply H to all points in image 2
   - Compute reprojection error for each point:
     ```
     error = ||src_point - H * dst_point||
     ```
   - If error < threshold (5.0 pixels), mark as inlier

4. **Model Evaluation**:
   - Count number of inliers for this hypothesis
   - More inliers = better model

5. **Iteration**:
   - Repeat steps 1-4 many times (2000 iterations here)
   - Each iteration produces a candidate model

6. **Best Model Selection**:
   - Select the model with the most inliers
   - This is the "consensus set"

7. **Final Estimation**:
   - Recompute homography using ALL inliers from the best model
   - Uses least squares for more accurate estimation

### Why RANSAC is Effective:

1. **Robust to Outliers**: Can handle up to 50% outliers in data
2. **Probability-Based**: More iterations = higher chance of finding correct model
3. **Simple Concept**: Easy to understand and implement

### Parameters Used:

- **ransacReprojThreshold=5.0**: Points with reprojection error < 5 pixels are inliers
- **maxIters=2000**: Maximum number of RANSAC iterations
- **confidence=0.995**: Probability that at least one sample is free from outliers

### Results Interpretation:

- **Inliers**: Correct matches used for final homography
- **Outliers**: Incorrect matches rejected by RANSAC
- **Inlier Ratio**: Higher ratio indicates better overall matching quality

6.Warp “tung_ping_chau_view2.jpeg” onto the other using the estimated transformation.
This can be done with the perspectiveTransform() and warpPerspective() functions in OpenCV.

In [ ]:
# Perspective Transformation and Image Stitching
print("="*60)
print("STEP 6: Warp Image 2 onto Image 1")
print("="*60)

# Get image dimensions
h1, w1 = img1.shape[:2]
h2, w2 = img2.shape[:2]

print(f"Image 1 dimensions: {w1} x {h1}")
print(f"Image 2 dimensions: {w2} x {h2}")

# Calculate the size of the stitched image
# Transform the four corners of image 2 to the coordinate system of image 1
corners2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
transformed_corners = cv2.perspectiveTransform(corners2, H)

print("\nOriginal corners of image 2 (x, y):")
for i, corner in enumerate(corners2.reshape(-1, 2)):
    print(f"  Corner {i}: ({corner[0]:.1f}, {corner[1]:.1f})")

print("\nTransformed corners of image 2 (in image 1 coordinate system):")
for i, corner in enumerate(transformed_corners.reshape(-1, 2)):
    print(f"  Corner {i}: ({corner[0]:.1f}, {corner[1]:.1f})")

# Find the bounding box of all points (image 1 corners + transformed image 2 corners)
all_corners = np.concatenate((
    np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2),
    transformed_corners
), axis=0)

[x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

print(f"\nBounding box of all corners:")
print(f"  Min: ({x_min}, {y_min})")
print(f"  Max: ({x_max}, {y_max})")

# Calculate translation to bring all points to positive coordinates
translation = [-x_min, -y_min]
H_translation = np.array([[1, 0, translation[0]],
                          [0, 1, translation[1]],
                          [0, 0, 1]])

# Apply translation to the homography matrix
H_final = H_translation @ H

print(f"\nTranslation vector: ({translation[0]}, {translation[1]})")

# Calculate stitched image dimensions
result_width = x_max - x_min
result_height = y_max - y_min
print(f"\nStitched image will be: {result_width} x {result_height} pixels")

# Transform the second image
print("\nWarping image 2 using the homography matrix...")
warped_img2 = cv2.warpPerspective(img2, H_final, (result_width, result_height))

# Create stitched image (place image 1 at correct position)
stitched_img = np.zeros((result_height, result_width, 3), dtype=np.uint8)
stitched_img[translation[1]:translation[1] + h1,
             translation[0]:translation[0] + w1] = img1

print(f"Image 1 placed at position: ({translation[0]}, {translation[1]})")

# Simple overlay blending
print("\nPerforming simple overlay blending...")
# Find non-black areas in warped_img2
mask_warped = cv2.cvtColor(warped_img2, cv2.COLOR_BGR2GRAY)
mask_warped = (mask_warped > 10).astype(np.uint8) * 255

# Copy warped_img2 to stitched_img using mask
stitched_img_simple = stitched_img.copy()
for c in range(3):
    stitched_img_simple[:, :, c] = stitched_img[:, :, c] * (1 - mask_warped/255) + \
                                   warped_img2[:, :, c] * (mask_warped/255)

# Calculate overlap area
overlap_mask = (mask_warped > 0) & (stitched_img.sum(axis=2) > 0)
overlap_area = np.sum(overlap_mask)
print(f"Overlap area between images: {overlap_area} pixels")

# Display intermediate results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Original images
axes[0, 0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Image 1 (Reference)')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Image 2 (To be warped)')
axes[0, 1].axis('off')

# Transformed image
axes[0, 2].imshow(cv2.cvtColor(warped_img2, cv2.COLOR_BGR2RGB))
axes[0, 2].set_title('Image 2 (After Warping)')
axes[0, 2].axis('off')

# Stitching canvas
axes[1, 0].imshow(cv2.cvtColor(stitched_img, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Canvas with Image 1 Placed')
axes[1, 0].axis('off')

# Mask
axes[1, 1].imshow(mask_warped, cmap='gray')
axes[1, 1].set_title('Mask for Warped Image 2')
axes[1, 1].axis('off')

# Simple stitching result
axes[1, 2].imshow(cv2.cvtColor(stitched_img_simple, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('Simple Stitching Result')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

7.Create the new image and show the result.

In [ ]:
# Final Image Stitching and Result
print("="*60)
print("STEP 7: Final Image Stitching")
print("="*60)

# Apply improved blending method (using the same function as before)
def improved_blending(img1, img2_warped):
    """Improved gradient blending method"""
    gray_warped = cv2.cvtColor(img2_warped, cv2.COLOR_BGR2GRAY)
    mask = (gray_warped > 10).astype(np.float32)

    kernel_size = 51
    mask_blurred = cv2.GaussianBlur(mask, (kernel_size, kernel_size), 0)
    mask_blurred = np.clip(mask_blurred, 0, 1)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img1_mask = (img1_gray > 10).astype(np.float32)

    overlap = (mask_blurred > 0) & (img1_mask > 0)

    if np.any(overlap):
        dist_transform = cv2.distanceTransform((img1_mask * 255).astype(np.uint8),
                                              cv2.DIST_L2, 5)
        dist_transform = dist_transform / (dist_transform.max() + 1e-6)
        mask_blurred[overlap] = dist_transform[overlap]

    mask_3ch = np.stack([mask_blurred, mask_blurred, mask_blurred], axis=2)
    result = img1.astype(np.float32) * (1 - mask_3ch) + img2_warped.astype(np.float32) * mask_3ch
    return result.astype(np.uint8)

# Create final stitched image
stitched_img_blended = improved_blending(stitched_img, warped_img2)

# Calculate basic quality metrics (optional)
def calculate_stitching_quality(stitched_img):
    gray = cv2.cvtColor(stitched_img, cv2.COLOR_BGR2GRAY)
    non_black_pixels = np.sum(gray > 10)
    total_pixels = gray.size
    coverage = non_black_pixels / total_pixels
    return coverage

coverage = calculate_stitching_quality(stitched_img_blended)
print(f"Image coverage: {coverage:.2%}")

# ============================================================
# DISPLAY AND SAVE FINAL RESULT (ONLY ONE IMAGE)
# ============================================================

# Display final stitched image
plt.figure(figsize=(14, 8))
plt.imshow(cv2.cvtColor(stitched_img_blended, cv2.COLOR_BGR2RGB))
plt.title('Final Stitched Panorama', fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

# Save the final image
output_filename = 'tung_ping_chau_panorama.jpg'
cv2.imwrite(output_filename, stitched_img_blended)
print(f"\n✓ Final image saved as: '{output_filename}'")
print(f"  - Dimensions: {stitched_img_blended.shape[1]} x {stitched_img_blended.shape[0]} pixels")
print(f"  - Format: BGR (OpenCV native)")

# Verify file was saved
import os
if os.path.exists(output_filename):
    file_size = os.path.getsize(output_filename) / 1024
    print(f"  - File size: {file_size:.1f} KB")

    # Optional: Verify by loading and checking dimensions
    saved_img = cv2.imread(output_filename)
    if saved_img is not None and saved_img.shape == stitched_img_blended.shape:
        print(f"  - Verification: File saved successfully and dimensions match")
    else:
        print(f"  - Verification: File saved but dimensions may differ due to compression")

print("\n" + "="*70)
print("IMAGE STITCHING ASSIGNMENT COMPLETE")
print("="*70)
print("\nSummary of steps completed:")
print("1. ✓ Images loaded and converted to grayscale")
print("2. ✓ Harris corner points detected")
print("3. ✓ SIFT keypoints extracted (limited to 5000)")
print("4. ✓ Keypoints matched using BFMatcher")
print("5. ✓ Homography matrix computed using RANSAC")
print("6. ✓ Image 2 warped onto Image 1")
print(f"7. ✓ Final panorama created and saved as '{output_filename}'")
print("\n" + "="*70)

Note: By default, OpenCV uses the BGR (Blue-Green-Red) color channel order, which is a historical legacy related to the BGR arrangement of early camera sensors. In contrast, Matplotlib and most image viewers use the RGB (Red-Green-Blue) order—a more modern and universal standard. This inconsistency in color channel ordering may occur, so a simple conversion might be necessary when outputting or saving the final stitched image.